# Is it reasonable to consider only largest mergers among many?
Multiple mergers can occur at a time. Especially when merger takes long, another merger can begin before the other ends. Then the effect of a merger can't be separated. In such case, I will take larger merger only. 
But how reliable is it? 

This script shows distribution of mergers and their mass ratio. 
1) Multiple mergers at a snapshot are rare. 

2) Multiple major mergers are even rarer.

3) Merger time overlap sometimes happen.

4) Merger time overlap among major ones are rare.

5) When the measurement window (~ 5-10 snapshots before and after) is considered, 
   many more overlap will be added.
   
   
!! The figure can be pickled and shown later quickly!.

In [222]:
def get_merger_info(main, atree, sat_root_idx,
                    dist_gal_scale_in=1.0,
                    dist_gal_scale_out=2.0):
    """
    Returns merger mass ratio and beginning of the merger. 
    nout_init_this_merger, mass_this_merger = get_merger_info()
    
    Assumes 
    """
    satellite = ctu.extract_main_tree(atree, sat_root_idx, no_subset=True)
    nout_min = max([min(main['nout']), min(satellite['nout'])])
    i_main_ok = (main['nout'] >= nout_min) * (main['nout'] <= max(satellite["nout"]))

    i_sat_ok = (satellite['nout'] >= nout_min)
    satellite = satellite[i_sat_ok]

    # distances at all valid nouts.
    dd = np.sqrt(np.square(main["x"][i_main_ok] - satellite['x']) \
               + np.square(main["y"][i_main_ok] - satellite['y']) \
               + np.square(main["z"][i_main_ok] - satellite['z'])) * 1e3
    rgal_tot = (main['rvir'][i_main_ok] + satellite['rvir'])
    #print(" Galaxy sizes : main {}, and the second {}, and the sum {}".format(
    #        main['r'][i_main_ok], satellite['r'], rgal_tot))
    #print(" dd :", dd)
    if sum(dist_gal_scale_in * rgal_tot > dd) > 0:
        # First close encounter is technically the beginning of merger,
        # but in practice that could be merely a flyby, 
        # and whether they will merger soon or not is not known. 
        # I can't call an encounter a merger if the encounter will end up merging in 100Gyrs.        
        #nout_init_this = min(satellite['nout'][dist_gal_scale * rgal_tot < dd])
        # First try
        # shouldn't go out 2Rgal.
        i_dist_bad = np.where(dist_gal_scale_out * rgal_tot < dd)[0]
        i_dist_ok = np.where(dist_gal_scale_in * rgal_tot > dd)[0]
        if len(i_dist_bad) > 0:
            i_dist_bad_last = min(i_dist_bad)
            i_dist_final = i_dist_ok[i_dist_ok < i_dist_bad_last]
        else:
            i_dist_final = i_dist_ok
        if len(i_dist_final) > 0:
            nout_init_this = satellite['nout'][min(i_dist_final)]
        
        
        # Second try
            mass_this = satellite['m'][satellite['nout'] == nout_init_this].squeeze()
        else:
            nout_init_this = -1
            mass_this = 0
        
    else:
        nout_init_this = -1
        mass_this = 0
    
    return nout_init_this, mass_this

In [223]:
def find_all_meger(alltrees, 
                   idx_all, 
                   nout_ini=37, 
                   dist_gal_scale=2,
                   min_mass_ratio = 0.01,
                   verbose=False,
                   do_plot = False):
    """
    Parameters
    ----------
    dist_gal_scale 
        if two galaxies are closer than dist_gal_scale * (sum of raidus of the two),
        that epoch is the nout_init_merger.
    nout_ini
        blabla
    """
    gal_list=[]
    mr_list=[]
    nout_list=[]
    nout_ini_list=[] # initial time when two halos(Galaxy stellar components in this case) overlap. 

    for idx in idx_all:
        # full tree of a galaxy
        atree = ctu.extract_a_tree(alltrees.data, idx)

        # main progenitor tree
        main = ctu.extract_main_tree(atree, idx)

        x_nout = main['nout'].flatten()
        i_nout_ok = x_nout > nout_ini
        main = main[i_nout_ok]
        #x_nout = x_nout[i_nout_ok]
        pos = np.zeros((3,len(main)))
        pos[0,:] = main['x']
        pos[1,:] = main['y']
        pos[2,:] = main['z']

        mass_ratios_this = []#np.zeros(len(main))
        nout_inits_this = []#np.zeros(len(main))
        nout_list_this = []
                
        for i, nout in enumerate(main['nout']):
            # merger ratio
            i_prgs = np.where(atree['desc_id'] == main['id'][i])[0]
            #print(" {} Progenitors at nout = {}".format(len(i_prgs), nout))
            # multiple prgs = merger
            if len(i_prgs) > 1:
                #if verbose: 
                #print("{}  {} Progenitors at nout = {}".format(idx, len(i_prgs), nout))
                id_prgs = atree['id'][i_prgs]
                mass_prgs = atree['m'][i_prgs]
                
                m_r = mass_prgs / max(mass_prgs)

                sats = id_prgs[mass_prgs < max(mass_prgs)]
                
                mass_ratios_now=[]
                nout_inits_now=[]
                
                for this_sat in sats:
                    n_i_t, mass_this_sat = get_merger_info(main, atree, this_sat,
                                                           dist_gal_scale_in=dist_gal_scale,
                                                           dist_gal_scale_out = 3.0)
                    mass_ratio = mass_this_sat / max(mass_prgs)
                    if mass_ratio > min_mass_ratio:
                        nout_inits_now.append(n_i_t)
                        mass_ratios_now.append(1./mass_ratio)
                
                nout_list_this.append(nout)
                nout_inits_this.append(nout_inits_now)
                mass_ratios_this.append(mass_ratios_now)

        #mr = 1./mass_ratios
        gal_list.append(idx)
        nout_list.append(nout_list_this)
        mr_list.append(mass_ratios_this)
        nout_ini_list.append(nout_inits_this)
    return gal_list, mr_list, nout_list, nout_ini_list

In [224]:
import tree.ctutils as ctu
import numpy as np
from analysis.misc import load_cat

# parameters used for lambda_arr clipping.
ind_upper = 20
ind_lower = 20
sig_upper = 2.0
sig_lower = 2.0

nout_ini = 62
nout_fi = 187

verbose=True
# In[4]:

base = './'
cdir = ['catalog/', 'easy/', 'catalog_GM/', "easy_final/"][3]

cluster = ['05427', '05420', '29172', \
           '29176', '10002', '36415',
           '06098', '39990', '36413','17891', '07206', '04466', '01605', '35663'][3]#[:-3]

wdir = base + cluster + '/'
alltrees = ctu.load_tree(wdir, is_gal=True)
ad = alltrees.data
tn = ad[ad['nout'] == nout_fi]

cat = load_cat(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle')
#idx_all = [tn['id'][tn['Orig_halo_id'] == id_final][0] for id_final in cat['id']]
idx_all = cat['idx'][cat["idx"] > 0].astype(int) # why idx are float???

Loaded an extended tree


In [225]:
gal_list, mr_list, nout_list, nout_init_list = \
                find_all_meger(alltrees, 
                               idx_all, 
                               nout_ini=37, 
                               dist_gal_scale=1,
                               min_mass_ratio = 0.001,
                               verbose=False,
                               do_plot = False)

In [41]:
import matplotlib.pyplot as plt

In [229]:
# simple scatter
fig, ax = plt.subplots()
for nout_list_this, mr_list_this in zip(nout_list, mr_list):
    for nout, mr in zip(nout_list_this, mr_list_this):
        for mm in mr:
            ax.scatter(nout, mm)
        try:
            ax.scatter(nout, mr[0], marker="*", color='r')
        except:
            #ax.scatter(nout, mr, marker="*", color='r')
            pass
    ax.set_yscale('log')
ax.set_ylabel("Merger mass ratio")
ax.set_xlabel("Nout")

plt.show()

In [219]:
import pickle

In [227]:
# merging process overlap
fig, ax = plt.subplots()
# each galaxy
for igal, (nout_init_this, nout_list_this, mr_list_this) in enumerate(zip(nout_init_list, nout_list, mr_list)):
    # each snapshot
    allnout_thisgal = []
    allmm_thisgal = []
    for nout_init, nout, mr in zip(nout_init_this, nout_list_this, mr_list_this):
        # each merger
        for ni, mm in zip(nout_init, mr):
            ax.plot([ni, nout], [mm,mm], 'y.-')
    #cm = ax.scatter(allnout_thisgal, igal * 10 + np.arange(len(allnout_thisgal)),
    #                s=1e3/np.array(allmm_thisgal))
               #c=np.array(allmm_thisgal), cmap="Greys", vmin=0, vmax=1e3)

ax.set_ylabel("mergers (arbitrary value)")
ax.set_xlabel("Nout")
#plt.colorbar(cm)
plt.show()


In [221]:
# merging process overlap
fig, ax = plt.subplots()
# each galaxy
for igal, (nout_init_this, nout_list_this, mr_list_this) in enumerate(zip(nout_init_list, nout_list, mr_list)):
    # each snapshot
    allnout_thisgal = []
    allmm_thisgal = []
    for nout_init, nout, mr in zip(nout_init_this, nout_list_this, mr_list_this):
        # each merger
        for ni, mm in zip(nout_init, mr):
            allnout_thisgal.append(nout)
            allnout_thisgal.append(ni)            
            allmm_thisgal.append(mm)
            allmm_thisgal.append(1e3) # one marker per one merger, and minimize the other one.
    #
    ax.plot(allnout_thisgal, igal * 10 + np.arange(len(allnout_thisgal)), '-')
    cm = ax.scatter(allnout_thisgal, igal * 10 + np.arange(len(allnout_thisgal)),
                    s=1e3/np.array(allmm_thisgal))
               #c=np.array(allmm_thisgal), cmap="Greys", vmin=0, vmax=1e3)

ax.set_ylabel("mergers (arbitrary value)")
ax.set_xlabel("Nout")
#plt.colorbar(cm)
pickle.dump(plt.gcf(), open("merger_overlap_plot.pickle", "wb"))
plt.show()

### A figure can be saved!

In [1]:
# On a separate ipython kernel, the following will generate the same figure!
import matplotlib.pyplot as plt
import pickle
fig = pickle.load(open("merger_overlap_plot.pickle", "rb"))
plt.show(fig)